In [1]:
!pip install openai langchain tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [2]:
!pip install python-dotenv


In [32]:
from google.colab import userdata
openai_api_key=userdata.get('OPENAI_API_KEY')

In [11]:
import langchain
import os
import openai
import sys
sys.path.append('../..')

In [19]:
!pip install PyPDF PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.3 MB/s eta 0:00:00


In [10]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter


In [20]:
import PyPDF2
def chunking(link,skip):
  loader = PyPDFLoader(link)
  pages = loader.load()
  r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2400,
    chunk_overlap=200,
    separators=["\n\n", "\n", "\. ", " ", ""])
  chunks=r_splitter.split_documents(pages[skip:])
  return chunks


In [21]:
pp=chunking("/content/EXata-8.1.1-InstallationGuide.pdf",2)

In [66]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain

# Schema
schema1 = {
    "properties": {
        "chapter_headings": {"type": "string"},
        "chapter_summary": {"type": "string"}
    },
    "required": ["chapter_headings", "chapter_summary"],
}
schema2 = {
    "properties": {
        "chapter_headings": {"type": "array", "items": {"type": "string"}},
        "chapter_summary": {"type": "array", "items": {"type": "string"}},
    },
    "required": ["chapter_headings", "chapter_summary"],
}
schema3={
    "properties": {
        "chapter_headings": {
            "type": "array",
            "items": {"type": "string"},
            "description": "An array of chapter headings or titles."
        },
        "chapter_summary": {
            "type": "array",
            "items": {"type": "string"},
            "description": "An array of chapter summaries with atleast 5 lines for the chapter."
        },
        "first_lines_of_chapters": {
            "type": "array",
            "items": {"type": "string"},
            "description": "An array of the first lines of each chapter with which each chapter starts."
        },
        "last_lines_of_chapters": {
            "type": "array",
            "items": {"type": "string"},
            "description": "An array of the last lines of each chapter with which chapter ends."
        }
    },
    "required": ["chapter_headings", "chapter_summary", "first_lines_of_chapters", "last_lines_of_chapters"]
}

schema4 = {
    "properties": {
        "chapter_headings": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "title": {"type": "string"},
                    "summary": {"type": "array", "items": {"type": "string"}},
                    "first_line": {"type": "array", "items": {"type": "string"}},
                    "last_line": {"type": "array", "items": {"type": "string"}}
                },
                "required": ["title", "summary", "first_line", "last_line"]
            },
            "description": "An array of chapter headings with associated summaries, first lines, and last lines."
        }
    },
    "required": ["chapter_headings"]
}

schema5 = {
    "properties": {
        "chapter_headings": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "chapter": {"type": "string"},
                    "summary": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "A brief array of chapter summaries starting from first-line and ending with last_line."
                    },
                    "first_line": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "The starting line of the chapter containing  the first 5 words"
                    },
                    "last_line": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "The ending line of the chapter containing the last 5 word"
                    }
                },
                "required": ["title", "summary", "first_line", "last_line"]
            },
            "description": "An array of chapter headings with associated summaries, first lines, and last lines."
        }
    },
    "required": ["chapter_headings"]
}
schema6 = {
    "properties": {
        "chapter_headings": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "title": {"type": "string"},
                    "summary": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "A brief array of title summaries."
                    },
                    "first_line": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "A single line from the start of the title where you found the title"
                    }
                },
                "required": ["title", "summary", "first_line"]
            }
        }
    },
    "required": ["chapter_headings"]
}


# Run chain
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0,openai_api_key=openai_api_key)
chain = create_extraction_chain(schema6, llm)

In [35]:
cc=chain.run(pp)

In [84]:
from pprint import pprint
pprint(cc)

[{'chapter_headings': [{'chapter': 'Installing EXata on Windows',
                        'first_line': ['EXata 8.1.1 Installation Guide  11 '
                                       'Installing EXata on Windows'],
                        'last_line': ['1.7  Uninstalling EXata and its '
                                      'Components'],
                        'summary': ['This chapter covers the system '
                                    'requirements and installation procedure '
                                    'for EXata on Windows.']}]},
 {'chapter_headings': [{'chapter': 'Installing EXata on Linux',
                        'first_line': ['EXata 8.1.1 Installation Guide  62 '
                                       'Installing EXata on Linux'],
                        'last_line': ['2.7  Uninstalling EXata and its '
                                      'Components'],
                        'summary': ['This chapter covers the system '
                                    'req

[{'chapter_headings': ['Supported Platforms',
                       'Downloading Connection Manager Installer',
                       'Installing Connection Manager on Windows',
                       'Installing Connection Manager on Linux',
                       'Testing Connection Manager'],
  'chapter_summary': ['EXata Connection Manager is supported on 64-bit (x86-64 '
                      'compatible) platforms running one of the following '
                      'operating systems: Windows 10 Home and Windows 10 Pro '
                      '64-bit editions, CentOS version 7.x and 8.x, Red Hat '
                      'Enterprise Linux (RHEL) version 7.x and 8.x, Ubuntu: '
                      'versions 18.04 LTS and 20.04 LTS',
                      'To download the Connection Manager installer for your '
                      'platform, go to the Keysight website and follow the '
                      'instructions.',
                      'To install Connection Manager on a Windows system, '
                      'download the installation package and follow the '
                      'installation prompts. The installer will also prompt '
                      'you to install the Npcap software.',
                      'To install Connection Manager on a Linux system, '
                      'download the installation package for your platform and '
                      'make sure it is executable. Then, run the installer '
                      'either from the GUI or from the command line.',
                      'To test the Connection Manager installation, start '
                      'Connection Manager on Windows by double-clicking the '
                      'icon or searching for it in the Windows search bar. On '
                      'Linux, open a command-line window and navigate to the '
                      'installation directory, then run the command to start '
                      'Connection Manager.'],
  'first_lines_of_chapters': ['EXata Connection Manager is supported on 64-bit '
                              '(x86-64 compatible) platforms running one of '
                              'the following operating systems:',
                              'To download the Connection Manager installer '
                              'for your platform, do the following:',
                              'This section describes how to install '
                              'Connection Manager on a Windows system.',
                              'This section describes how to install '
                              'Connection Manager on a Linux system.',
                              'To test the Connection Manager installation on '
                              'Windows, start Connection Manager by doing one '
                              'of the following:'],
  'last_lines_of_chapters': ['To test Connection Manager installation on '
                             'Linux, start Connection Manager by doing the '
                             'following:',
                             'The following startup screen is displayed when '
                             'Connection Manager starts:']}]

In [36]:
pp1=chunking("/content/EXata-8.1.1-ReleaseNotes.pdf",2)

In [37]:
cc1=chain.run(pp1[:10])

In [39]:
from pprint import pprint
pprint(cc1)

[{'chapter_headings': [{'chapter': 'EXata 8.1.1 Release Notes',
                        'first_line': ['EXata 8.1.1 Release Notes',
                                       'This section lists release notes for '
                                       'EXata 8.1.1 which are described in the '
                                       'following tables:'],
                        'last_line': ['TABLE 6.\u2002\u2002Limitations and '
                                      'Known Issues for EXata 8.1.1'],
                        'summary': ['This section lists release notes for '
                                    'EXata 8.1.1 which are described in the '
                                    'following tables:',
                                    '•Table  1: New Features for EXata 8.1.1',
                                    '•Table  2: Updated and Enhanced Features '
                                    'for EXata 8.1.1',
                                    '•Table  3: Features Removed from EXat

In [67]:
pp2=chunking("/hist.pdf",5)
cc2=chain.run(pp2[:10])

In [68]:
pprint(cc2)

[{'chapter_headings': [{'first_line': ['not the only bodies to travel round '
                                       'the sun. There are also the planets, '
                                       'Mercury and Venus, at distances of',
                                       'If, then, we represent our earth as a '
                                       'little ball of one inch diameter, the '
                                       'sun would be a big globe nine feet '
                                       'across',
                                       'The nearest star to earth on this '
                                       'scale would be 40,000 miles away.'],
                        'summary': ['There are planets such as Mercury, Venus, '
                                    'Mars, Jupiter, Saturn, Uranus, and '
                                    'Neptune that travel around the sun at '
                                    'different distances.',
                                   